# Spark GraphX Analysis of US Flight Data

In this notebook, you will have the opportunity to explore capabilities of GraphX while analyzing airline data. 

The code required to execute each task is contained in the Markdown cells. Cut and paste this code into the empty Code cells below the Markdown cell and run it.
  

![GraphX](https://spark.apache.org/docs/latest/img/graphx_logo.png)

## Import required libraries
    val sqlContext = new org.apache.spark.sql.SQLContext(sc)
    import sqlContext.implicits._
    import org.apache.spark.sql.Row;
    import sys.process._
    import org.apache.spark.rdd.RDD
    import org.apache.spark.graphx._
    import org.apache.spark.graphx.util.GraphGenerators

## Download data as CSV file
#### Data from the US Bureau of Transportation Statistics (BTS) for March 2016 required to run this notebook is hosted at https://ibm.box.com/shared/static/rfhew36plv01cixjweq1mrdrpib4fbcx.csv.
#### Download the dataset to local storage.
 
    "rm -f airport_Mar_2016.csv".!
    "wget -q -O airport_Mar_2016.csv https://ibm.box.com/shared/static/rfhew36plv01cixjweq1mrdrpib4fbcx.csv".!

## Read the CSV data file and clean the data
#### The data is in CSV format and contains a header row.<br>Notice that the DataFrame contains a blank column. This is due to the fact that the CSV file contains a trailing comma at the end of each row.<br>Drop the blank column and any rows that contain nulls.
    val airport_df_raw = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load("airport_Mar_2016.csv")
    val airport_df = airport_df_raw.drop("").na.drop
    airport_df_raw.show(10)

## Convert the DataFrame to an RDD
#### To accomplish use a custom case class rather than the generic Row object.
GraphX is based on RDDs, which is why we must decompose the DataFrame into an RDD.
    
    case class flight(origin_id: Int, 
                  origin: String, 
                  origin_city: String, 
                  dest_id: Int, 
                  dest: String, 
                  dest_city: String, 
                  elapsed_time: Double, 
                  distance: Double)
    val flights= airport_df.map {case Row(origin_id: Int, 
                                      origin: String, 
                                      origin_city: String, 
                                      dest_id: Int, 
                                      dest: String, 
                                      dest_city: String, 
                                      elapsed_time: Double, 
                                      distance: Double) => flight(origin_id = origin_id, 
                                                                  origin = origin, 
                                                                  origin_city = origin_city, 
                                                                  dest_id = dest_id, 
                                                                  dest = dest, 
                                                                  dest_city = dest_city, 
                                                                  elapsed_time = elapsed_time, 
                                                                  distance = distance)}
    flights.take(5).foreach(println)


## Extract airport ID and airport codes, which will be used to create the graph vertices
#### Each vertex is keyed by a unique 64-bit long identifier (VertexID), so the airport identifier must be converted to Long. The airport code is used as the vertex property (VertexProperty). VertexID can occur only once in the VertexRDD.
    val airportVertices = flights.map(x => (x.origin_id.toLong, x.origin)).distinct()
    airportVertices.take(5).foreach(println)
    println("The number of airports used in the analysis = " + airportVertices.count())

## Extract origin airport id, destination airport id and distance between airports for use as graph edges
#### Edges have a srcId and a dstId corresponding to the source and destination vertex identifiers. In addition, the Edge class has an attr member which stores the edge property. srcid and dstid must be Long, so they must be converted during the mapping.
    val tripEdges = flights.map(x => Edge(x.origin_id, x.dest_id, x.distance))
    tripEdges.takeSample(false, 5).foreach(println)
    println("The number of trips used in the analysis = " + tripEdges.count())

## Create the graph
#### The Graph class represents a graph with arbitrary objects associated with vertices and edges. The graph provides basic operations to access and manipulate the data associated with vertices and edges as well as the underlying structure.
    val graph = Graph(airportVertices, tripEdges)

## Show the vertices of the graph
#### Deconstruct a graph into the respective vertex and edge views by using the graph.vertices and graph.edges members respectively.
    graph.vertices.take(5).foreach(println)
    println("Number of airports = " + graph.vertices.count())
    println("Number of airports = " + graph.numVertices)

## Show the edges of the graph, which represent flights
#### using the edge member of the graph class
    graph.edges.take(5).foreach(println)
    println("Number of trips = " + graph.edges.count())
    println("Number of trips = " + graph.numEdges)

## Determine the number of flights greater than 4000 miles using the Edge class attr member which stores the edge property
    val distanceInput = 4000
    val number_trips_over_option2 = graph.edges.filter (e => e.attr > distanceInput).count
    println("Number of trips over " + distanceInput + " miles = " + number_trips_over_option2)

## In addition to the vertex and edge views of the property graph, GraphX also exposes a triplet view
<img src="https://raw.githubusercontent.com/bradenrc/Spark_POT/master/Modules/GraphX/triplet.png" height="70%" width="70%"></img>
#### The triplet view logically joins the vertex and edge properties yielding an EdgeTriplet object containing instances of the EdgeTriplet class.
#### View the graph using the triplet view
    graph.triplets.takeSample(false, 5).foreach(println)

## Show graph triplets for flights over 4000 miles
#### Not surprisingly all the resulting trips represent flights to/from Hawaii.
    val distanceInput = 4000
    graph.triplets.filter(x => x.attr > distanceInput).distinct().sortBy(x => x.attr, false).take(10).foreach(println)

## Show graph triplets originating from Boston
#### Sort the result in descending order of distance.
    graph.triplets.filter(x => x.srcAttr == "BOS").distinct().sortBy(x => x.attr, false).take(5).foreach(println)

## Determine the number of flights from Boston to San Francisco in March 2016
#### Keep in mind that the dataset contains only March 2016 data
    
    println(" The number of flights from Boston To San Francisco in March 2016 = " + 
        graph.triplets.filter(x => x.srcAttr == "BOS").filter(x => x.dstAttr == "SFO").count())

## A common aggregation task is computing the degree of each vertex
#### The degree of a vertrix represents the number of edges adjacent to each vertex. In the context of directed graphs it is often useful to know the in-degree, out-degree, and the total degree of each vertex. 
## Define a reduce operation and compute the highest degree vertex
    def max(a: (VertexId, Int), b: (VertexId, Int)): (VertexId, Int) = {
      if (a._2 > b._2) a else b
    }

    val maxInDegree  = graph.inDegrees.reduce(max)
    val maxOutDegree = graph.outDegrees.reduce(max)
    val maxDegrees = graph.degrees.reduce(max)

    println("Max in-degree = " + maxInDegree)
    println("Max out-degree = " + maxOutDegree)
    println("Max total degrees = " + maxDegrees)

## Find which airport corresponds to the airport id with the maximum number of degrees
    val maxDegrees_id = maxDegrees._1
    val airport = graph.triplets.filter(x => x.srcId == maxDegrees_id).map(x => x.srcAttr).first()
    println("The airport with the most inbound and outbound flights is " + airport +".")

## PageRank measures the importance of each vertex in a graph
<img src="https://raw.githubusercontent.com/bradenrc/Spark_POT/master/Modules/GraphX/pagerank.png" height="70%" width="70%"></img>
#### PageRank assumes an edge from u to v represents an endorsement of v’s importance by u. For example, if a Twitter user is followed by many others, the user will be ranked highly.
#### PageRank works by counting the number of edges to a vertex to determine a rough estimate of how important the vertex is. The underlying assumption is that more important vertices are likely to have more edges from other vertices. 
## Calculate PageRanks in descending order
#### Get the airport codes by joining the vertex id to the airport code in the airportVertices RDD
    val ranks = graph.pageRank(0.1).vertices
    ranks.sortBy(x => x._2, false).take(5).foreach(println)

    val ranksByAirport = ranks.join(airportVertices).distinct().sortBy(x => x._2._1, false)
    println("The most important airports by PageRank are")
    ranksByAirport.map(x => x._2._2).take(5).foreach(println)

## Subgraphs
The subgraph operator takes vertex and edge predicates and returns the graph containing only the vertices that satisfy the vertex predicate and edges that satisfy the edge predicate and connect vertices that satisfy the vertex predicate. The subgraph operator can be used in number of situations to restrict the graph to the vertices and edges of interest or eliminate broken links.
## Create a subgraph consisting of only flights that are greater than 4000 miles using an edge predicate to create the subgraph.
    val distanceInput = 4000
    val subgraph = graph.subgraph(epred = x => x.attr > distanceInput)
    println("Created subgraph only with flights that are greater than " + distanceInput + " miles apart.")

    subgraph.edges.distinct().sortBy(x => x.attr, true).take(5).foreach(println)
    println("The number of vertices in the subgraph = "+ subgraph.numVertices)
    println("The number of edges in the subgraph = "+ subgraph.numEdges)

## Create a subgraph using a vertex predicate to  include only the following airports - BOS, SFO, ATL, LAX, ORD
    val airportList = List("BOS", "SFO", "ATL", "LAX", "ORD")
    val subgraph2 = graph.subgraph(vpred = (vid, attr) => airportList contains attr)
    subgraph2.vertices.take(10).foreach(println)

## Triangle Counting
#### A vertex is part of a triangle when it has two adjacent vertices with an edge between them. GraphX implements a triangle counting algorithm in the TriangleCount object that determines the number of triangles passing through each vertex, providing a measure of clustering. Note that TriangleCount requires the edges to be in canonical orientation (srcId < dstId) and the graph to be partitioned using the partitionBy member of the graph class. partitionBy repartitions the edges in the graph according to partitionStrategy. 
## Create a subgraph for Triangle Counting
#### To meet the edge orientation requirement, create a new subgraph that only contains edges/flights where org_id < dest_id. Although not exact, you can roughly view this as working with round trip segments starting at the lower order airport id. 

#### Note that the org_id is less then the dest_id for the sample Edges shown. Also, note that the number of trips/edges is about half the number of edges in the original graph, while the number of vertices remains the same as we only emplyed an edge predicate.

    val subgraphTC = graph.subgraph(epred = x => x.srcId < x.dstId)
    println("The number of vertices in the subgraph = "+ subgraphTC.numVertices)
    println("The number of edges in the subgraph = "+ subgraphTC.numEdges)
    subgraphTC.edges.takeSample(false, 5).foreach(println)

## Create a Traingle Count graph
#### Show that the edges of this graph have the same attribute (distance) as the original graph and subgraph.

    val TriangleCountGraph = subgraphTC.partitionBy(PartitionStrategy.RandomVertexCut).triangleCount()
    val TriangleCountEdges = TriangleCountGraph.edges
    TriangleCountEdges.sortBy(x => x.attr, false).take(5).foreach(println)

## Determine the airports by airport code in Triangle Count descending order
#### To get the airport codes, join the vertex id to the airport code in the airportVertices RDD

    val TriangleCountVertices = TriangleCountGraph.vertices
    println("The airports with the highest Triangle Clount are")
    TriangleCountVertices.join(airportVertices).sortBy(x => x._2._1, false).take(5).foreach(println)

## Conclusion
In this notebook you saw how to use Apache Spark GraphX to analyze airline data that is readily downloadable from the US Bureau of Transportation website. You explored the Graph abstraction introduced by GraphX and used several of the basic operators to analyze the strucuture and relationships in the data. You saw how GraphX exposes RDD views of the vertices and edges stored within the graph and how GraphX maintains the vertices and edges in optimized data structures, VertrexRDD and EdgeRDD, which provide additional functionality above and beyond standard Spark RDDs. 

You also employed graph operators to transform the graphs into new graphs with transformed properties and structure. Finally, you worked with a few of the graph algorithms included with GraphX, like PageRank and Triangle Counting, to demonstrate how they simplify analytics tasks. 